In [1]:
from tensorflow import keras
import ipywidgets as widgets
from IPython.display import display, Image
import numpy as np
import io
import requests

In [2]:
import warnings;
warnings.filterwarnings('ignore');

In [3]:
path = keras.utils.get_file(
    "emily_char_model.h5", origin='https://github.com/katarinagresova/M7DataSP_2020/blob/master/assignment_07/models/emily_char_model.h5?raw=true'
)
model2 = keras.models.load_model(path)

1843200/1835024 [==============================] - 0s 0us/step


In [4]:
path = keras.utils.get_file(
    "emily-together.txt", origin="https://raw.githubusercontent.com/katarinagresova/M7DataSP_2020/main/assignment_07/data/emily-together.txt"
)
with io.open(path, encoding="utf-8") as f:
    text = f.read().lower()

chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

188416/184901 [==============================] - 0s 0us/step


In [5]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [6]:
def generate_poem(seed):
  diversity = 0.5
  generated = ""
  maxlen = 20

  pad_len = maxlen - len(seed)
  if pad_len < 0:
    sentence = seed[:maxlen]
  elif pad_len > 0:
      sentence = " " * (pad_len - 1) + seed + "." 

  for i in range(200):
      x_pred = np.zeros((1, maxlen, len(chars)))
      for t, char in enumerate(sentence):
          x_pred[0, t, char_indices[char]] = 1.0
      preds = model2.predict(x_pred, verbose=0)[0]
      next_index = sample(preds, diversity)
      next_char = indices_char[next_index]
      sentence = sentence[1:] + next_char
      generated += next_char

  # remove last unfinished line
  last_enter = generated.rfind('\n')
  generated = generated[:last_enter]

  return generated

In [11]:
img =Image(
    url='https://raw.githubusercontent.com/katarinagresova/M7DataSP_2020/master/assignment_07/graphics/poetry.jpg',
    format='jpg',
    width=460,
    height=400,
)

entering = widgets.Text(
    value='sunshine',
    placeholder='Type something',
    description='Your word:',
    disabled=False
)
button = widgets.Button(description="Generate poem")
output = widgets.Output()


def on_button_clicked(b):
    with output:
      output.clear_output()
      print("Please wait, poem is beeing written.")
      poem = generate_poem(entering.get_interact_value())
      output.clear_output()
      print(poem)
button.on_click(on_button_clicked)

In [12]:
display(img)
display(widgets.VBox([widgets.HBox([entering, button]), output]))
